In [125]:
import pandas as pd
import math
# price : 제품을 얻기 위해 지불하는 가격
# retail_price : 시장 또는 다른 상점/장소에서 유사한 물품에 대한 참조 가격. 
# currency_buyer : 현재 가격
# units_sold : 판매된 개수
# rating : 평균제품등급
# rating_count : 제품의 총 등급 개수
# badges_count : 제품 또는 판매자가 보유한 배지 수
# badge_local_product : 지역 제품이냐 아니냐 (1이면 지역제품)
# tags : 판매자가 붙인 꼬리표
# product_color : 제품 색
# countries_shipped_to : 이 제품이 배송되는 국가 수
# merchant_rating_count : 이 판매자의 등급 수
# merchant_rating : 상인의 등급.
# product_id : 제품 식별자
# urgency_text : 검색 결과의 일부 제품 위에 나타나는 텍스트 배너
#              - Réduction sur les achats en gros : 도매 구매 할인
#              - Quantité limitée : 제한 수량

summer_products=pd.read_csv("./data/summer-products-with-rating-and-performance_2020-08.csv")
summer_products_df=summer_products[["price","units_sold","rating","rating_count","merchant_rating_count","urgency_text"]]
summer_products_df.head()

,price,units_sold,rating,rating_count,merchant_rating_count,urgency_text
0,16.00,100,3.76,54,568,Quantité limitée !
1,8.00,20000,3.45,6135,17752,Quantité limitée !
2,8.00,100,3.57,14,295,Quantité limitée !
3,8.00,5000,4.03,579,23832,NaN
4,2.72,100,3.10,20,14482,Quantité limitée !


In [126]:
# 데이터 전처리
summer_products_df["urgency_text"]=summer_products_df["urgency_text"].fillna("-")
# drop_duplicates() : 중복 행이 제거 된 DataFrame을 반환한다.
urgency_text=summer_products_df["urgency_text"].drop_duplicates().to_numpy()

# urgency_text 데이터 라벨링 
encoder={}
for i,text in enumerate(urgency_text):
    encoder[text]=i

<ipython-input-126-7ed6ba8e8ed5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summer_products_df["urgency_text"]=summer_products_df["urgency_text"].fillna("-")


In [127]:
# 데이터 스케일링
summer_products_df["urgency_num"]=summer_products_df["urgency_text"].apply(lambda x:encoder[x])
summer_products_df=summer_products_df.drop("urgency_text",axis=1)

summer_products_df["units_sold"]=summer_products_df["units_sold"].apply(lambda x: math.sqrt(x)/10)

summer_products_df["merchant_rating_count"]=summer_products_df["merchant_rating_count"].apply(lambda x:math.sqrt(x)/10)

summer_products_df["rating_count"]=summer_products_df["rating_count"].apply(lambda x:math.sqrt(x)/5)
summer_products_df

<ipython-input-127-a4ce7d6a0531>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summer_products_df["urgency_num"]=summer_products_df["urgency_text"].apply(lambda x:encoder[x])


,price,units_sold,rating,rating_count,merchant_rating_count,urgency_num
0,16.00,1.000000,3.76,1.469694,2.383275,0
1,8.00,14.142136,3.45,15.665248,13.323663,0
2,8.00,1.000000,3.57,0.748331,1.717556,0
3,8.00,7.071068,4.03,4.812484,15.437616,1
4,2.72,1.000000,3.10,0.894427,12.034118,0
...,...,...,...,...,...,...
1568,6.00,10.000000,4.08,7.394593,7.291090,1
1569,2.00,1.000000,3.07,1.058301,6.659580,0
1570,5.00,1.000000,3.71,1.536229,1.449138,1
1571,13.00,1.000000,2.50,0.282843,0.556776,1


- SVM(Support Vector Machine)\
    : 이항 분류 문제를 위해 만들졌다
    <img src="./image/SVM.png" height="400px" width="400px" >
    
    - 클래스들이 선형경계에 의해 분류될 수 있는 데이터에만 사용가능
    - 결정 경계 : 클래스들을 구분하기 위한 선
    - 서포트 벡터 : 결정 경계 근처의 가장 가까운 포인터들
    - 마진(Margin) : 결정 경계와 서포트 벡터사이의 거리
    - 파라미터
        - C : 데이터 샘픔들이 다른 클래스에 놓이는 것을 허용하는 정도 
        - kernel : 주어진 데이터를 고차원 특징 공간으로 사상(결정 경계 정의 모형)\
        (linear, plotnormaial, RBF)
        - gamma : 결정 경계의 곡률을 결정
- SVM 종류
    - SVR(Support Vector Regression)
        - 회귀에 적용하는 방법
            - svm 회귀는 제한된 마진 오류(즉, 도로 밖의 샘플)안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습한다
            - 도로 폭은 epsilon 하이퍼파라미터로 조절
    - SVC(Support Vector Classifiction)
        - 일정한 마진 오류 안에서, 두 클래스 간의 도로 폭이 가능한 한 최대가 되도록 한다\
        
        


In [94]:
from sklearn.svm import SVR # 비선형 회귀 작업 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

x=summer_products_df.to_numpy()[:,1:]
y=summer_products_df["price"].to_numpy()
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print(x.shape,y.shape)

# 모델 생성
svr=SVR(C=0.3,max_iter=1000,gamma="auto")

# 모델 학습
svr.fit(X_train,y_train)

# 모델 예측
predict_y=svr.predict(X_test)

# 평균 제곱오차
round(mean_squared_error(y_test, predict_y),4)

#  support vectors의 좌표 확인
# np.round(svr.support_vectors_,3)

# for i,y in enumerate(predict_y):
#     print("y_test :",np.round(y_test[i],4),"  ","predict : ",np.round(y,4))
[{y_test[i]:y} for i,y in enumerate(predict_y)]

(1573, 5) (1573,)


[{6.0: 8.63700688081077},
 {8.0: 8.007457843951643},
 {13.0: 8.28068207887536},
 {7.0: 7.226541320873055},
 {4.67: 7.549375783043506},
 {4.0: 7.907660905460305},
 {7.0: 7.65902160549592},
 {6.0: 8.234267133901263},
 {9.0: 8.066328409996908},
 {3.0: 7.5970214261611595},
 {20.0: 7.6029771757259565},
 {7.0: 7.552852013187547},
 {6.0: 7.651222140352123},
 {14.0: 8.506455636642814},
 {4.86: 8.123780057340019},
 {5.0: 7.981600806585466},
 {12.0: 8.03409663613587},
 {9.0: 7.180487394244031},
 {8.0: 6.765964448639503},
 {6.0: 7.44518525923605},
 {8.0: 7.955359891469103},
 {12.0: 8.321298070578948},
 {1.0: 7.890997423832245},
 {11.0: 8.375009824847394},
 {3.0: 8.074159176147267},
 {9.0: 6.477505567217783},
 {9.0: 8.351682574939439},
 {5.0: 6.767475934247743},
 {9.0: 6.568326972550902},
 {11.0: 6.8961454518440215},
 {13.0: 7.541979670569045},
 {9.0: 8.279583668473654},
 {7.0: 6.900567571925786},
 {6.0: 7.389667328399604},
 {15.0: 8.758127090334982},
 {8.0: 7.741295588861666},
 {16.0: 6.370357799

In [95]:
from sklearn.svm import SVC

x=summer_products_df.to_numpy()[:,:-1]
y=summer_products_df["urgency_num"].to_numpy()
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print(x.shape,y.shape)

# 모델 생성
scv=SVC(kernel="sigmoid",max_iter=100)

# 모델 학습
scv.fit(X_train,y_train)

# 모델 예측
predict_y=scv.predict(X_test)

# support vectors의 좌표 확인
# np.round(scv.support_vectors_,3)
[{y_test[i]:y} for i,y in enumerate(predict_y)]

# 평균 제곱오차
round(mean_squared_error(y_test, predict_y),4)

(1573, 5) (1573,)


C:\Users\hyebin\anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


0.4127

In [117]:
from sklearn.svm import LinearSVC

# 모델 생성
linescv=LinearSVC(C=0.3,random_state=11,max_iter=100).fit(X_train,y_train)

# 
linescv_score=linescv.score(X_train,y_train)

result=linescv.predict(X_test)

#[{y_test[i]:y} for i,y in enumerate(result)]

# 평균 제곱오차
round(mean_squared_error(y_test, predict_y),4)


C:\Users\hyebin\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.4127

In [123]:
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5)).fit(X_train,y_train)
result=regr.predict(X_test)
[{y_test[i]:y} for i,y in enumerate(result)]
round(mean_squared_error(y_test, result),4)

0.2984

In [ ]:
#  https://analysis-flood.tistory.com/96 
# https://bskyvision.com/163 
# http://hleecaster.com/ml-svm-concept/#comment-785 
# https://jeongmin-lee.tistory.com/87